<a href="https://colab.research.google.com/github/Abhiramiraju003/RFSVM/blob/main/RFSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy  scikit-learn tensorflow transformers fasttext matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313473 sha256=de3dd1cfb57e414584815e14bf20c2d3d52d8d3d3fc582fad0bf2bd59f7e13be
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import fasttext


In [ ]:
df=pd.read_csv('/content/1429_1.csv')

<ipython-input-3-42c3cc30ba5a>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/1429_1.csv')


In [ ]:
import nltk # Import the nltk library
nltk.download('stopwords')
nltk.download('punkt_tab')  # Download the punkt_tab data package
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import nltk # Import the nltk library
nltk.download('stopwords')
nltk.download('punkt_tab')  # Download the punkt_tab data package
stop_words = set(nltk.corpus.stopwords.words('english'))

def preprocess_text(text):
    # Check if the text is a string before applying lowercasing
    if isinstance(text, str):
        text = text.lower()
        text = nltk.word_tokenize(text)
        text = [word for word in text if word not in stop_words]
        return ' '.join(text)
    else:
        # Handle non-string values (e.g., NaN)
        return ""  # Or any other appropriate handling
# Print the columns of your DataFrame to see the actual column names
print(df.columns)

# Assuming the actual column name containing text data is 'review_text'
df['reviews.text'] = df['reviews.text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['reviews.text'], df['reviews.rating'], test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

# Load dataset
df = pd.read_csv("/content/1429_1.csv")
X = df['reviews.text'].astype(str)  # Convert reviews to string
y = df['reviews.rating']

# Convert ratings to binary sentiment (0 = Negative, 1 = Positive)
y_binary = np.where(y <= 2, 0, 1)

# **Fix 1: Balance the dataset** (Oversample if necessary)
positive_samples = np.sum(y_binary == 1)
negative_samples = np.sum(y_binary == 0)

if positive_samples > negative_samples:
    positive_indices = np.where(y_binary == 1)[0]
    negative_indices = np.where(y_binary == 0)[0]

    np.random.shuffle(positive_indices)
    positive_indices = positive_indices[:negative_samples]  # Match negative samples

    balanced_indices = np.concatenate([positive_indices, negative_indices])
    np.random.shuffle(balanced_indices)

    X = X.iloc[balanced_indices]
    y_binary = y_binary[balanced_indices]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# **Fix 2: Improved TF-IDF Vectorizer (removes stop words & limits features)**
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# **Fix 3: Optimize Model Parameters**
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
svm = SVC(kernel='linear', probability=True, random_state=42)  # Enable probability for VotingClassifier

rf_svm = VotingClassifier(estimators=[('rf', rf), ('svm', svm)], voting='soft')  # Soft voting for better prediction

# Train the model
rf_svm.fit(X_train_tfidf, y_train)

# Evaluate the Model
y_pred_rf_svm = rf_svm.predict(X_test_tfidf)

# **Fix 4: Print Metrics Clearly**
print("Model Evaluation Metrics:")
accuracy = accuracy_score(y_test, y_pred_rf_svm)
precision = precision_score(y_test, y_pred_rf_svm)
recall = recall_score(y_test, y_pred_rf_svm)
f1 = f1_score(y_test, y_pred_rf_svm)

print(f" RFSVM Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred_rf_svm)
print("\nConfusion Matrix:")
print(conf_matrix)

# **Fix 5: Additional Performance Metrics**
tp = conf_matrix[1, 1]  # True Positives
tn = conf_matrix[0, 0]  # True Negatives
fp = conf_matrix[0, 1]  # False Positives
fn = conf_matrix[1, 0]  # False Negatives

specificity = tn / (tn + fp)
fpr = fp / (fp + tn)
fnr = fn / (fn + tp)

print(f" Specificity: {specificity:.4f}")
print(f" False Positive Rate: {fpr:.4f}")
print(f" False Negative Rate: {fnr:.4f}")

# **Fix 6: Handle User Input Properly**
while True:
    user_input = input("\n Please enter a review (or type 'exit' to quit): ").strip()

    if user_input.lower() == 'exit':
        print("👋 Exiting...")
        break

    if not user_input:
        print("⚠️ Empty input! Please enter a valid review.")
        continue

    # Convert user input to TF-IDF vector
    user_input_tfidf = vectorizer.transform([user_input])

    # Predict sentiment using the trained model
    y_pred_user = rf_svm.predict(user_input_tfidf)
    y_pred_prob = rf_svm.predict_proba(user_input_tfidf)[0][1]  # Get probability of being positive

    print(f"Predicted Sentiment: {'🟢 Positive' if y_pred_user == 1 else '🔴 Negative'} (Confidence: {y_pred_prob:.4f})")


<ipython-input-9-8517e05b3940>:10: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/1429_1.csv")


Model Evaluation Metrics:
 RFSVM Accuracy: 0.8308
Precision: 0.8713
Recall: 0.8187
F1 Score: 0.8442

Confusion Matrix:
[[121  22]
 [ 33 149]]
 Specificity: 0.8462
 False Positive Rate: 0.1538
 False Negative Rate: 0.1813

 Please enter a review (or type 'exit' to quit): this product is awesome
Predicted Sentiment: 🟢 Positive (Confidence: 0.7675)

 Please enter a review (or type 'exit' to quit): exit
👋 Exiting...
